# How to get historical data from the blockchain (Sepolia)

Library: [web3.py](https://web3py.readthedocs.io/en/stable/index.html)

Some other useful resources: [Etherscan(Sepolia)](https://docs.etherscan.io/v/sepolia-etherscan/), [Etherscan](https://docs.etherscan.io/), [alchemy](https://docs.alchemy.com/)

If you want to implement some other functions, please refer to above documentations.

In [18]:
from web3 import Web3, HTTPProvider
import json
import time
import requests
import os

### Set up your address and provider

directly paste your url/address/private key into the code is not safe. You can store those information in the local environment and use `os.environ.get('PRIVATE_KEY')` instead.

In [7]:
w3 = Web3(HTTPProvider(os.environ.get("PROVIDER_URI_SEPOLIA")))
my_address = os.environ.get("ADDRESS")
private_key = os.environ.get("PRIVATE_KEY")
address = Web3.to_checksum_address(my_address)

### Get some on-chain information

In [8]:
# get a block by number
block = w3.eth.get_block(12345)

# get the latest block
block = w3.eth.get_block('latest')

# convert between wei and eth
wei = w3.to_wei(1, 'ether')
eth = w3.from_wei(1000000000000000000, 'ether')

# get a transaction by its hash
transaction = w3.eth.get_transaction('0xeb7d89660746178954f8295ece884465a7a9f3947f152b25421ccab567b4ffce')

# get a transaction's receipt
receipt = w3.eth.get_transaction_receipt('0xeb7d89660746178954f8295ece884465a7a9f3947f152b25421ccab567b4ffce')

# get the balance of an address
balance = w3.eth.get_balance(address)

# get the nonce of an address
nonce = w3.eth.get_transaction_count(address)

# get the gas price
gas_price = w3.eth.gas_price

# get the current block number
block_number = w3.eth.block_number

### Get information from etherscan

In [10]:
def get_account_balance(address):
    api_key = os.environ.get("ETHERSCAN_API_KEY")
    url = f"https://api-sepolia.etherscan.io/api?module=account&action=balance&address={address}&tag=latest&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    if data['status'] == '1':
        balance = int(data['result']) / 10**18
        return balance
    else:
        return "Error: " + data['message']

print(get_account_balance(my_address))

48.93486310911901


In [24]:
def get_account_transactions(address):
    api_key = os.environ.get("ETHERSCAN_API_KEY")
    url = f"https://api-sepolia.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&page=1&offset=100&sort=asc&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    if data['status'] == '1':
        txs = data['result']
        return txs
    else:
        return "Error: " + data['message']

transactions = get_account_transactions(my_address)
tx = transactions[0]
# print the first transaction in the list
# this includes all the fields returned by the API
print(tx['blockNumber'])
print(tx['timeStamp'])
print(tx['hash'])
print(tx['from'])
print(tx['to'])
print(tx['value'])
print(tx['gas'])
print(tx['gasPrice'])
print(tx['cumulativeGasUsed'])
print(tx['gasUsed'])
print(tx['txreceipt_status'])
print(tx['input'])
print(tx['contractAddress'])

4977317
1703816328
0xa4fd9e936b0d743cd62c67f5f55fae1ea4f9dd1a0c88853f26f9f0a9be5bb97e
0xa7e4ef0a9e15bdef215e2ed87ae050f974ecd60b
0x3dad7c5704a6ea567a0549ee711768c0a405d06c
500000000000000000
63000
15249514035
42000
21000
1
0x



### Specify a contract you want to collect data from

For example, you want to collect data from [this contract](https://sepolia.etherscan.io/address/0x7d164F30f0b6a2ABAE55Adae9645a22268747D61#code).

You can get the contract's address directly from the above link, and get the contract's abi [here](https://sepolia.etherscan.io/address/0x7d164F30f0b6a2ABAE55Adae9645a22268747D61#code). Use the contract address and the abi to instantiate a contract.

In [32]:
contract_address = "0x7d164F30f0b6a2ABAE55Adae9645a22268747D61"
contract_abi = json.loads('[{"inputs":[{"internalType":"address","name":"_owner","type":"address"},{"internalType":"address","name":"_collector","type":"address"},{"internalType":"uint256","name":"_chainId","type":"uint256"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"account","type":"address"}],"name":"Paused","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"account","type":"address"}],"name":"Unpaused","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"fromAsset","type":"address"},{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint64","name":"toChainId","type":"uint64"},{"indexed":false,"internalType":"bytes","name":"toAddress","type":"bytes"},{"indexed":false,"internalType":"uint256","name":"net","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"fee","type":"uint256"}],"name":"WrapperLock","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"fromAsset","type":"address"},{"indexed":true,"internalType":"bytes","name":"txHash","type":"bytes"},{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"efee","type":"uint256"}],"name":"WrapperSpeedUp","type":"event"},{"inputs":[],"name":"chainId","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"extractFee","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"feeCollector","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"fromAsset","type":"address"},{"internalType":"uint16","name":"toChainId","type":"uint16"},{"internalType":"bytes","name":"toAddress","type":"bytes"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"fee","type":"uint256"}],"name":"lock","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"lockProxy","outputs":[{"internalType":"contract ILockProxy","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"pause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"paused","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_collector","type":"address"}],"name":"setFeeCollector","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_lockProxy","type":"address"}],"name":"setLockProxy","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"fromAsset","type":"address"},{"internalType":"bytes","name":"txHash","type":"bytes"},{"internalType":"uint256","name":"fee","type":"uint256"}],"name":"speedUp","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"unpause","outputs":[],"stateMutability":"nonpayable","type":"function"}]')
contract = w3.eth.contract(address=contract_address, abi=contract_abi)

### Get the logs of the contract

From the contract's abi or the source code, you can know which event you want to listen to. For example, in the above contract, there is a **WrapperLock** event. You can get all the emitted **WrapperLock** events in this way:

In [31]:
# Get all the logs for the contract
# You can specify the block range with fromBlock and toBlock
logs = contract.events.WrapperLock().get_logs(fromBlock=0x457CFA, toBlock=0x45CA43)

# If you don't want all of them, you can manually add a filter
# For example, only get the logs with a specific sender
logs = contract.events.WrapperLock().get_logs(fromBlock=0x457CFA, toBlock=0x45CA43, argument_filters={"sender": "0x00468F6fe26DBAb138571f2C3B91c8dFE3E241Dc"})

### Listen to an event in real time

In [ ]:
# define a filter 
event_filter = contract.events.WrapperLock.create_filter(fromBlock='latest')

# Listen for events
def handle_event(event):
    print(event)
    # Do something with the event

def log_loop(event_filter, poll_interval):
    while True:
        for event in event_filter.get_new_entries():
            handle_event(event)
        time.sleep(poll_interval)

log_loop(event_filter, 2)

In [30]:
# Or you can use threading
from threading import Thread

stop_thread = False

def handle_event(event):
    print(event)
    # Do something with the event

def log_loop(event_filter, poll_interval):
    while not stop_thread:
        for event in event_filter.get_new_entries():
            handle_event(event)
        time.sleep(poll_interval)

worker = Thread(target=log_loop, args=(event_filter, 2), daemon=True)
worker.start()
time.sleep(60)
stop_thread = True
worker.join()

AttributeDict({'args': AttributeDict({'fromAsset': '0x0000000000000000000000000000000000000000', 'sender': '0x041026C06210595f5700c7c83db05b9FfdBF53c9', 'toChainId': 2, 'toAddress': b'aleo1mjehxw4l84pp4vs47ll8p9gh44yzcgwq8mdsxws0v32rkzs9ug9q5ug5wk', 'net': 10000000000000000, 'fee': 10000000000000000}), 'event': 'WrapperLock', 'logIndex': 121, 'transactionIndex': 102, 'transactionHash': HexBytes('0x4d714e170fb550c4ceaf4e81c4082b26d9c33a21f18097d4f7841e10370bb13f'), 'address': '0x7d164F30f0b6a2ABAE55Adae9645a22268747D61', 'blockHash': HexBytes('0x43bc70184558a102628f33c091ccd394e1d80d290ac02f4ea1216d77a7fad2e0'), 'blockNumber': 5262825})
AttributeDict({'args': AttributeDict({'fromAsset': '0x0000000000000000000000000000000000000000', 'sender': '0x1741f9E80ADcdDF913897c5e7952C0129951b5a9', 'toChainId': 2, 'toAddress': b'aleo1d8qfuw7z7rqux5kmud7s6zjqn3wmkaqjx93uy33e6qazdq8cu59svvwlvt', 'net': 40000000000000000, 'fee': 10000000000000000}), 'event': 'WrapperLock', 'logIndex': 44, 'transaction